In [3]:
import os
import ee
import pandas as pd
from multiprocessing import Pool
from datetime import timedelta
import datetime
import json
import rioxarray
import xarray
import numpy as np

In [4]:
os.getcwd()

'/ccvi_stefano'

In [5]:

def downloadCDS():

    import cdsapi

    c = cdsapi.Client(url="https://cds.climate.copernicus.eu/api/v2", key="xx")

    c.retrieve(
        'reanalysis-era5-single-levels-monthly-means',
        {
            'format': 'netcdf',
            'product_type': 'monthly_averaged_reanalysis',
            'variable': [
                '2m_temperature', 'total_precipitation', 'land_sea_mask'
            ],
            'year': [
                '2022', '2023',
            ],
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'time': '00:00',
        },
        'download.nc')
    
    


def downloadCDS_forecast():
    import cdsapi

    c = cdsapi.Client(url="https://cds.climate.copernicus.eu/api/v2",  key="xx")


    c.retrieve(
        'seasonal-monthly-single-levels',
        {
            'format': 'netcdf',
            'originating_centre': 'ecmwf',
            'system': '51',
            'variable': [
                '2m_temperature', 'total_precipitation',
            ],
            'product_type': 'monthly_mean',
            'year': '2023',
            'month': '12',
            'leadtime_month': [
                '1', '2', '3',
                '4', '5', '6',
            ],
        },
        'download_forecast.nc')
    
def download_forecast_anomaly():
    import cdsapi

    c = cdsapi.Client(url="https://cds.climate.copernicus.eu/api/v2", key="xx")

    c.retrieve(
        'seasonal-postprocessed-single-levels',
        {
            'format': 'netcdf',
            'originating_centre': 'ecmwf',
            'system': '51',
            'variable': [
                '2m_temperature_anomaly', 'total_precipitation_anomalous_rate_of_accumulation',
            ],
            'product_type': 'monthly_mean',
            'year': '2023',
            'month': '12',
            'leadtime_month': [
                '1', '2', '3',
                '4', '5', '6',
            ],
        },
        'download_anomaly.nc')

def shiftCoo(xar):
    
    xar.coords['longitude'] = (xar.coords['longitude'] + 180.0) % 360.0 - 180.0
    xar = xar.sortby(xar.longitude)
    return xar

In [6]:
downloadCDS()

2023-12-07 09:57:46,075 INFO Welcome to the CDS
2023-12-07 09:57:46,075 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2023-12-07 09:57:46,300 INFO Request is queued
2023-12-07 09:57:48,863 INFO Request is running
2023-12-07 09:58:07,278 INFO Request is completed
2023-12-07 09:58:07,279 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data9/adaptor.mars.internal-1701943077.2667-23185-11-16092e36-1546-486d-a1b8-597de4b8f9b1.nc to download.nc (273.3M)
2023-12-07 09:58:14,235 INFO Download rate 39.3M/s


In [7]:
downloadCDS_forecast()

2023-12-07 09:58:14,320 INFO Welcome to the CDS
2023-12-07 09:58:14,321 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2023-12-07 09:58:14,358 INFO Request is queued
2023-12-07 09:58:15,389 INFO Request is running
2023-12-07 09:58:27,709 INFO Request is completed
2023-12-07 09:58:27,710 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data0/adaptor.mars.external-1701943103.9133828-25270-11-144565ae-0f36-4518-a02c-70d1c6bc6d68.nc to download_forecast.nc (75.6M)
2023-12-07 09:58:29,244 INFO Download rate 49.3M/s


In [8]:
download_forecast_anomaly()

2023-12-07 09:58:29,308 INFO Welcome to the CDS
2023-12-07 09:58:29,308 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-postprocessed-single-levels
2023-12-07 09:58:29,372 INFO Request is queued
2023-12-07 09:58:30,403 INFO Request is running
2023-12-07 09:58:50,350 INFO Request is completed
2023-12-07 09:58:50,351 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data1/adaptor.mars.external-1701943123.9008255-8610-4-bbfbf42d-57d4-45ec-9e21-bd8fac11bba0.nc to download_anomaly.nc (75.6M)
2023-12-07 09:58:53,048 INFO Download rate 28.1M/s


In [12]:
xds = xarray.open_dataset(f"download.nc")

xds_forecast = xarray.open_dataset(f"download_forecast.nc")
xds_forecast = xds_forecast.rename(name_dict={"tprate":"tp"})
xds_forecast["tp"]=xds_forecast["tp"]*86400

#xds_forecast = xds_forecast.interp(latitude=xds.latitude, longitude=xds.longitude)

xds = xds.interp(latitude=xds_forecast.latitude, longitude=xds_forecast.longitude)

xds_anomaly = xarray.open_dataset(f"download_anomaly.nc")


In [13]:
xds.time

<xarray.DataArray 'time' (time: 23)>
array(['2022-01-01T00:00:00.000000000', '2022-02-01T00:00:00.000000000',
       '2022-03-01T00:00:00.000000000', '2022-04-01T00:00:00.000000000',
       '2022-05-01T00:00:00.000000000', '2022-06-01T00:00:00.000000000',
       '2022-07-01T00:00:00.000000000', '2022-08-01T00:00:00.000000000',
       '2022-09-01T00:00:00.000000000', '2022-10-01T00:00:00.000000000',
       '2022-11-01T00:00:00.000000000', '2022-12-01T00:00:00.000000000',
       '2023-01-01T00:00:00.000000000', '2023-02-01T00:00:00.000000000',
       '2023-03-01T00:00:00.000000000', '2023-04-01T00:00:00.000000000',
       '2023-05-01T00:00:00.000000000', '2023-06-01T00:00:00.000000000',
       '2023-07-01T00:00:00.000000000', '2023-08-01T00:00:00.000000000',
       '2023-09-01T00:00:00.000000000', '2023-10-01T00:00:00.000000000',
       '2023-11-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2022-01-01 2022-02-01 ... 2023-11-01
Attributes:
    long_name:  time

In [14]:
xds_forecast.time

<xarray.DataArray 'time' (time: 6)>
array(['2023-12-01T00:00:00.000000000', '2024-01-01T00:00:00.000000000',
       '2024-02-01T00:00:00.000000000', '2024-03-01T00:00:00.000000000',
       '2024-04-01T00:00:00.000000000', '2024-05-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2023-12-01 2024-01-01 ... 2024-05-01
Attributes:
    long_name:  time

In [15]:
xds_anomaly.time

<xarray.DataArray 'time' (time: 6)>
array(['2023-12-01T00:00:00.000000000', '2024-01-01T00:00:00.000000000',
       '2024-02-01T00:00:00.000000000', '2024-03-01T00:00:00.000000000',
       '2024-04-01T00:00:00.000000000', '2024-05-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2023-12-01 2024-01-01 ... 2024-05-01
Attributes:
    long_name:  time

In [16]:
xds = xds.mean(dim="expver")

In [17]:

xds_forecast = xds_forecast.mean(dim="number")

In [18]:
xds_anomaly  = xds_anomaly.mean(dim="number")

In [19]:
#mask = xds["lsm"].sel(time=slice('2022-01-01'))
xds = xds.drop_vars("lsm")


In [20]:
xds = xarray.concat([xds, xds_forecast], dim="time")

In [21]:
xds = shiftCoo(xds)
xds_forecast = shiftCoo(xds_forecast)
xds_anomaly = shiftCoo(xds_anomaly)

In [22]:
list_of_values=[]

for time in xds.time.values:
    list_of_values.append(xds["t2m"].sel(time=slice(time)).mean().values.item())
list_of_values


[277.2901306152344,
 277.16925048828125,
 277.1742858886719,
 277.3507080078125,
 277.826171875,
 278.4115905761719,
 278.8658752441406,
 279.1646423339844,
 279.278564453125,
 279.27142333984375,
 279.1353454589844,
 279.0151062011719,
 278.87066650390625,
 278.7268981933594,
 278.6270751953125,
 278.5979919433594,
 278.6603088378906,
 278.812744140625,
 278.9721984863281,
 279.098876953125,
 279.19146728515625,
 279.2027893066406,
 279.1780090332031,
 279.1240539550781,
 279.05340576171875,
 278.9801330566406,
 278.9220275878906,
 278.9152526855469,
 278.9622497558594]

In [23]:
list_of_values=[]

for time in xds.time.values:
    list_of_values.append(xds["tp"].sel(time=slice(time)).mean().values.item())
list_of_values


[0.0023485078747430242,
 0.002352736927640774,
 0.0023621228505443367,
 0.0023530234685699442,
 0.0023620508374174896,
 0.0023811547201866507,
 0.0024055850979702864,
 0.0024198841562862567,
 0.0024292034978993667,
 0.0024319983582690727,
 0.0024250541664957875,
 0.0024206411601868246,
 0.0024135512916936388,
 0.0024092018437437135,
 0.0024021994466011877,
 0.0023956048174367437,
 0.0023951106812394396,
 0.00239765162843047,
 0.002404945729016868,
 0.002411600397099212,
 0.002416576650705427,
 0.002416398522835256,
 0.0024157396445095326,
 0.0024155557590476114,
 0.002416054493360732,
 0.002416504698723666,
 0.002416961566410497,
 0.0024180214170301953,
 0.002420737880474011]

In [24]:
#xds["tp"] = xds["tp"]*mask
#xds["t2m"] = xds["t2m"]*mask

In [25]:
xds.time

<xarray.DataArray 'time' (time: 29)>
array(['2022-01-01T00:00:00.000000000', '2022-02-01T00:00:00.000000000',
       '2022-03-01T00:00:00.000000000', '2022-04-01T00:00:00.000000000',
       '2022-05-01T00:00:00.000000000', '2022-06-01T00:00:00.000000000',
       '2022-07-01T00:00:00.000000000', '2022-08-01T00:00:00.000000000',
       '2022-09-01T00:00:00.000000000', '2022-10-01T00:00:00.000000000',
       '2022-11-01T00:00:00.000000000', '2022-12-01T00:00:00.000000000',
       '2023-01-01T00:00:00.000000000', '2023-02-01T00:00:00.000000000',
       '2023-03-01T00:00:00.000000000', '2023-04-01T00:00:00.000000000',
       '2023-05-01T00:00:00.000000000', '2023-06-01T00:00:00.000000000',
       '2023-07-01T00:00:00.000000000', '2023-08-01T00:00:00.000000000',
       '2023-09-01T00:00:00.000000000', '2023-10-01T00:00:00.000000000',
       '2023-11-01T00:00:00.000000000', '2023-12-01T00:00:00.000000000',
       '2024-01-01T00:00:00.000000000', '2024-02-01T00:00:00.000000000',
       '2024-03-01T00:00:00.000000000', '2024-04-01T00:00:00.000000000',
       '2024-05-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2022-01-01 2022-02-01 ... 2024-05-01
Attributes:
    long_name:  time

In [26]:
import pandas as pd
df=[]
for time in xds.time.values:
    current_time=pd.to_datetime(time)
    quarter = current_time.quarter
    year = current_time.year
    df.append([time,current_time.strftime('%Y-%m-%d'),f"{year}Q{quarter}"])
df = pd.DataFrame(df)
df.columns=["time","current_time","quarter"]
df

,time,current_time,quarter
0,2022-01-01,2022-01-01,2022Q1
1,2022-02-01,2022-02-01,2022Q1
2,2022-03-01,2022-03-01,2022Q1
3,2022-04-01,2022-04-01,2022Q2
4,2022-05-01,2022-05-01,2022Q2
5,2022-06-01,2022-06-01,2022Q2
6,2022-07-01,2022-07-01,2022Q3
7,2022-08-01,2022-08-01,2022Q3
8,2022-09-01,2022-09-01,2022Q3
9,2022-10-01,2022-10-01,2022Q4


In [28]:
if not os.path.exists("OUTPUT/ANOMALY"):
    os.makedirs("OUTPUT/ANOMALY")
if not os.path.exists("OUTPUT/QUARTERS/DELTA"):
    os.makedirs("OUTPUT/QUARTERS/DELTA")
if not os.path.exists("OUTPUT/QUARTERS/DELTAPERCENTAGE"):
    os.makedirs("OUTPUT/QUARTERS/DELTAPERCENTAGE")

In [29]:

for quarter in df.quarter.unique():
    date_list = df.loc[df.quarter == quarter]["current_time"].values.tolist()
    
    t2m  = xds["t2m"].sel(time=date_list).mean(dim="time")
    tp  = xds["tp"].sel(time=date_list).mean(dim="time")
    
    t2m.rio.to_raster(f'OUTPUT/QUARTERS/output_t2m_{quarter}.tif')
    tp.rio.to_raster(f'OUTPUT/QUARTERS/output_tp_{quarter}.tif')


In [30]:
date_list_ref = df.loc[df.quarter == "2022Q1"]["current_time"].values.tolist()
t2m_ref_Q1  = xds["t2m"].sel(time=date_list_ref).mean(dim="time")
tp_ref_Q1  = xds["tp"].sel(time=date_list_ref).mean(dim="time")

date_list_ref = df.loc[df.quarter == "2022Q2"]["current_time"].values.tolist()
t2m_ref_Q2  = xds["t2m"].sel(time=date_list_ref).mean(dim="time")
tp_ref_Q2 = xds["tp"].sel(time=date_list_ref).mean(dim="time")

date_list_ref = df.loc[df.quarter == "2022Q3"]["current_time"].values.tolist()
t2m_ref_Q3  = xds["t2m"].sel(time=date_list_ref).mean(dim="time")
tp_ref_Q3 = xds["tp"].sel(time=date_list_ref).mean(dim="time")

date_list_ref = df.loc[df.quarter == "2022Q4"]["current_time"].values.tolist()
t2m_ref_Q4  = xds["t2m"].sel(time=date_list_ref).mean(dim="time")
tp_ref_Q4 = xds["tp"].sel(time=date_list_ref).mean(dim="time")

refs={
    "t2m_ref_Q1":t2m_ref_Q1,
    "t2m_ref_Q2":t2m_ref_Q2,
    "t2m_ref_Q3":t2m_ref_Q3,
    "t2m_ref_Q4":t2m_ref_Q4,
    "tp_ref_Q1":tp_ref_Q1,
    "tp_ref_Q2":tp_ref_Q2,
    "tp_ref_Q3":tp_ref_Q3,
    "tp_ref_Q4":tp_ref_Q4
}



for quarter in df.quarter.unique():
    
    if (int(quarter[:4])>2022):
        date_list = df.loc[df.quarter == quarter]["current_time"].values.tolist()

        t2m  = xds["t2m"].sel(time=date_list).mean(dim="time")
        tp  = xds["tp"].sel(time=date_list).mean(dim="time")

        quarternum = quarter[-1]
        t2m_delta = t2m - refs[f"t2m_ref_Q{quarternum}"]
        tp_delta = tp - refs[f"tp_ref_Q{quarternum}"]

        t2m_delta.rio.to_raster(f'OUTPUT/QUARTERS/DELTA/output_t2m_delta_{quarter}.tif')
        tp_delta.rio.to_raster(f'OUTPUT/QUARTERS/DELTA/output_tp_delta_{quarter}.tif')

In [31]:
for quarter in df.quarter.unique():
    
    if (int(quarter[:4])>2022):
        date_list = df.loc[df.quarter == quarter]["current_time"].values.tolist()

        t2m  = xds["t2m"].sel(time=date_list).mean(dim="time")
        tp  = xds["tp"].sel(time=date_list).mean(dim="time")

        quarternum = quarter[-1]
        t2m_delta = (t2m - refs[f"t2m_ref_Q{quarternum}"])/refs[f"t2m_ref_Q{quarternum}"]
        tp_delta = (tp - refs[f"tp_ref_Q{quarternum}"])/refs[f"tp_ref_Q{quarternum}"]

        t2m_delta.rio.to_raster(f'OUTPUT/QUARTERS/DELTAPERCENTAGE/output_t2m_deltapercentage_{quarter}.tif')
        tp_delta.rio.to_raster(f'OUTPUT/QUARTERS/DELTAPERCENTAGE/output_tp_deltapercentage_{quarter}.tif')

In [32]:
list_of_values=[]

for time in xds_anomaly.time.values:
    date = pd.to_datetime(time)
    date = date.strftime('%Y-%m-%d')
    xds_anomaly["tpara"].sel(time=time).rio.to_raster(f'OUTPUT/ANOMALY/Total_precipitation_anomalous_rate_of_accumulation_{date}.tif')


In [33]:
list_of_values=[]

for time in xds_anomaly.time.values:
    date = pd.to_datetime(time)
    date = date.strftime('%Y-%m-%d')
    xds_anomaly["t2a"].sel(time=time).rio.to_raster(f'OUTPUT/ANOMALY/2m_temperature_anomaly_{date}.tif')
